In [ ]:
import os
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as T

class LostAndFoundObstacleDataset(Dataset):
    def __init__(self, root, img_transform=None, mask_transform=None):
        """
        Dataset for LostAndFound evaluation.
        Contains only object-level binary masks; class masks are not used.

        Args:
            root (str): root path to LostAndFound dataset
            img_transform (callable): transform applied to RGB images
            mask_transform (callable): transform applied to binary masks
        """
        self.img_dir = os.path.join(root, 'leftImg8bit')
        self.gt_dir = os.path.join(root, 'gtCoarse')
        self.img_transform = img_transform
        self.mask_transform = mask_transform

        self.img_paths = []
        self.mask_paths = []

        # Traverse subfolders (each sequence is in its own folder)
        for sequence in os.listdir(self.img_dir):
            img_seq_dir = os.path.join(self.img_dir, sequence)
            gt_seq_dir = os.path.join(self.gt_dir, sequence)
            for fname in os.listdir(img_seq_dir):
                if fname.endswith('_leftImg8bit.png'):
                    self.img_paths.append(os.path.join(img_seq_dir, fname))
                    label_fname = fname.replace('leftImg8bit', 'gtCoarse_labelIds')
                    self.mask_paths.append(os.path.join(gt_seq_dir, label_fname))

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        """
        Returns:
            dict: {
                'image': RGB input tensor,
                'object_mask': binary mask (1 = unknown object),
                'class_mask': tensor filled with 255 (ignored),
                'is_ood': True
            }
        """
        # Load RGB image
        image = Image.open(self.img_paths[idx]).convert('RGB')

        # Load coarse label mask
        mask = Image.open(self.mask_paths[idx])
        mask_np = np.array(mask)

        # In LostAndFound, obstacle pixels have label == 1 (others == 0 or 255)
        object_mask = (mask_np == 1).astype(np.uint8)
        object_mask = torch.from_numpy(object_mask).long()

        # Dummy class mask (ignored in evaluation)
        class_mask = torch.full_like(object_mask, fill_value=255)

        # Apply transforms
        if self.img_transform:
            image = self.img_transform(image)
        else:
            image = T.ToTensor()(image)

        if self.mask_transform:
            object_mask = self.mask_transform(object_mask.unsqueeze(0)).squeeze(0)
            class_mask = self.mask_transform(class_mask.unsqueeze(0)).squeeze(0)

        return {
            'image': image,
            'class_mask': class_mask,      # ignored (255)
            'object_mask': object_mask,
            'is_ood': True
        }
